In [5]:
import nautilus_trader as nt
from nautilus_trader.model.data import BarType, BarSpecification
from nautilus_trader.model.objects import Money, Price, Quantity
from nautilus_trader.model.identifiers import InstrumentId, Venue
from nautilus_trader.model.currencies import USD
# LIGNE CORRIGÉE: 'Side' est remplacé par 'OrderSide'
from nautilus_trader.model.enums import AccountType, OmsType, OrderSide
from nautilus_trader.backtest.engine import BacktestEngine, BacktestEngineConfig
from nautilus_trader.trading.strategy import Strategy, StrategyConfig
from nautilus_trader.test_kit.providers import TestInstrumentProvider

# 1. Définir une stratégie "Buy and Hold" très simple
class BuyAndHold(Strategy):
    def on_start(self):
        # S'abonner aux données de barres de 1 minute
        self.subscribe_bars(
            BarType(self.instrument.id, BarSpecification(1, 'MIN'))
        )
        
    def on_bar(self, bar: nt.Bar):
        # Si nous n'avons pas de position, acheter 0.1 BTC
        if self.portfolio.positions.is_empty():
            print(f"[{bar.ts_event}] Première barre détectée, achat de 0.1 BTC...")
            
            # Créer un ordre au marché
            # LIGNE CORRIGÉE: 'Side.BUY' est remplacé par 'OrderSide.BUY'
            buy_market = self.order_factory.market(
                instrument_id=self.instrument.id,
                quantity=Quantity(0.1, self.instrument.precision),
                side=OrderSide.BUY,
            )
            
            # Soumettre l'ordre
            self.submit_order(buy_market)

# 2. Configuration du moteur de Backtest
config_moteur = BacktestEngineConfig(
    trader_id="BACKTEST-001",
    oms_type=OmsType.NETTING, # Mode de gestion des positions simple
    account_type=AccountType.MARGIN,
)
engine = BacktestEngine(config=config_moteur)

# 3. Définir le marché (Venue) et l'Instrument
VENUE = Venue("SIM-EXCHANGE")
# Utilise un instrument de test fourni par le 'test_kit' de Nautilus
instrument = TestInstrumentProvider.btcusd_testnet_spot(VENUE)
engine.add_instrument(instrument)

# 4. Configurer et ajouter la stratégie
config_strategie = StrategyConfig(
    strategy_id="BuyAndHold-001",
    instrument_id=instrument.id,
)
engine.add_strategy(BuyAndHold(config=config_strategie))

# 5. Ajouter des données simulées (pour cet exemple)
# Dans un cas réel, nous chargerions des données Parquet ou CSV ici
print("Génération de données simulées...")
engine.add_data(
    nt.test_kit.bars_generator(
        instrument_id=instrument.id,
        start=nt.to_datetime_nanos("2024-01-01 00:00:00"),
        end=nt.to_datetime_nanos("2024-01-02 00:00:00"),
        bar_spec=BarSpecification(1, "MIN"),
        price_start=30000.0, # Prix de départ simulé
    )
)

# 6. Exécuter le backtest
print("Démarrage du backtest...")
results = engine.run()
print("Backtest terminé.")

# 7. Afficher les résultats
print("\n--- Résultats ---")
print(f"Profit et Perte (PnL) final : {results.portfolio_metrics.total_pnl.normalize()}")
print(f"Nombre total de transactions : {results.portfolio_metrics.total_trades}")

# Afficher les positions fermées
if not results.positions_closed:
    print("Aucune position n'a été fermée.")
else:
    print(results.positions_closed[0])

AttributeError: module 'nautilus_trader' has no attribute 'Bar'